# 世界の最新の感染者数

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
del df["Lat"]
del df["Long"]

In [3]:
df.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,65728,66275,66903,67743,68366,69130,70111,70761,71838,72977
1,NaN,Albania,0,0,0,0,0,0,0,0,...,132209,132215,132229,132244,132264,132285,132297,132309,132315,132337
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,126860,127107,127361,127646,127926,128198,128456,128725,128913,129218
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,13569,13569,13664,13671,13682,13693,13693,13693,13727,13729
4,NaN,Angola,0,0,0,0,0,0,0,0,...,32441,32623,32933,33338,33607,33944,34180,34366,34551,34752


In [4]:
latest_date = df.tail(1).columns[-1]
latest_date

'6/1/21'

In [5]:
latest_column_key = df.columns[-1]
latest_column_key2 = df.columns[-2]
df = df[['Province/State','Country/Region',latest_column_key2,latest_column_key]]
df = df.rename(columns={latest_column_key:"latest_confirmed"})
df = df.rename(columns={latest_column_key2:"latest_confirmed2"})
df["new_confirmed"] = df.latest_confirmed - df.latest_confirmed2
df.head()

,Province/State,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
0,NaN,Afghanistan,71838,72977,1139
1,NaN,Albania,132315,132337,22
2,NaN,Algeria,128913,129218,305
3,NaN,Andorra,13727,13729,2
4,NaN,Angola,34551,34752,201


## 全世界の最新の感染者数

In [6]:
print("全世界感染者数({}時点):{:,}人".format(latest_date,df["latest_confirmed"].sum()))
print("全世界感染者数平均({}時点):{:,}人".format(latest_date,df["latest_confirmed"].mean()))
print("全世界の新規感染者数({}時点):{:,}人".format(latest_date,df["new_confirmed"].sum()))
print("全世界の新規感染者数平均({}時点):{:,}人".format(latest_date,df["new_confirmed"].mean()))


全世界感染者数(6/1/21時点):171,187,600人
全世界感染者数平均(6/1/21時点):620,244.9275362319人
全世界の新規感染者数(6/1/21時点):463,388人
全世界の新規感染者数平均(6/1/21時点):1,678.9420289855072人


## 国別の最新の感染者数

In [7]:
country = df.groupby(["Country/Region"],as_index=False).sum("latest_confirmed")
country = country.sort_values("latest_confirmed", ascending=False)

In [8]:
country.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
179,US,33267507,33290450,22943
79,India,28175044,28307832,132788
23,Brazil,16545554,16624480,78926
62,France,5728788,5738641,9853
178,Turkey,5249404,5256516,7112


In [9]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [10]:
plt.figure(figsize=(9.5,5))
plt.title("Worst 50")
plt.xticks(rotation="90",fontsize=9)
plt.ylabel("count (1000 people)")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.bar(country.head(50)["Country/Region"],country.head(50)["latest_confirmed"]/1000,label="positive")
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

## ヒストグラム 

In [11]:
plt.figure(figsize=(10,5))
plt.title("histgram(million)")
#plt.xticks(rotation="90",fontsize=9)
#plt.ylabel("count (1000 people)")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.hist(country["latest_confirmed"]/1000000,bins=20)

<IPython.core.display.Javascript object>

(array([173.,   8.,   6.,   3.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.]),
 array([1.00000000e-06, 1.66452345e+00, 3.32904590e+00, 4.99356835e+00,
        6.65809080e+00, 8.32261325e+00, 9.98713570e+00, 1.16516582e+01,
        1.33161806e+01, 1.49807031e+01, 1.66452255e+01, 1.83097480e+01,
        1.99742704e+01, 2.16387929e+01, 2.33033153e+01, 2.49678378e+01,
        2.66323602e+01, 2.82968827e+01, 2.99614051e+01, 3.16259276e+01,
        3.32904500e+01]),
 <a list of 20 Patch objects>)

## 人口当たりの分析

In [12]:
pop = pd.read_excel("world-data/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx",header=16)
pop.head(1)

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950,1951,1952,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Estimates,WORLD,NaN,900,World,0,2536431.018,2584034.227,2630861.69,...,7041194.168,7125827.957,7210582.041,7295290.759,7379796.967,7464021.934,7547858.9,7631091.113,7713468.205,7794798.729


In [13]:
pop.iloc[:,-1]

0      7794798.729
1              ...
2      1273304.261
3      6521494.468
4      1057438.163
          ...     
284         62.273
285      37742.157
286         56.772
287          5.795
288     331002.647
Name: 2020, Length: 289, dtype: object

In [14]:
pop_latest = pop.loc[pop["Type"]=="Country/Area"][[pop.columns[2],pop.columns[-1]]]

In [15]:
pop_latest.columns = ["country","population"]

In [16]:
pop_latest.head()

,country,population
26,Burundi,11890.781
27,Comoros,869.595
28,Djibouti,988.002
29,Eritrea,3546.427
30,Ethiopia,114963.583


In [17]:
country.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
179,US,33267507,33290450,22943
79,India,28175044,28307832,132788
23,Brazil,16545554,16624480,78926
62,France,5728788,5738641,9853
178,Turkey,5249404,5256516,7112


### 世界全体での人口10万人あたり陽性者数

In [18]:
country.latest_confirmed.sum()

171187600

In [19]:
pop_latest.population.sum()

7794798.729

In [20]:
print("****世界全体での人口10万人あたり陽性者数:{}****".format(country.latest_confirmed.sum() / (pop_latest.population.sum() / 100)))

****世界全体での人口10万人あたり陽性者数:2196.177296574812****


### 各国の人口１０万人あたり陽性者数

In [21]:
c_merged = pd.merge(country, pop_latest, left_on="Country/Region", right_on="country", how="left")
c_merged.loc[c_merged["country"].isna()]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population
0,US,33267507,33290450,22943,NaN,NaN
5,Russia,5013512,5022881,9369,NaN,NaN
12,Iran,2913136,2923823,10687,NaN,NaN
51,Bolivia,371279,374718,3439,NaN,NaN
59,West Bank and Gaza,308350,308732,382,NaN,NaN
67,Moldova,255186,255241,55,NaN,NaN
72,Venezuela,234165,235567,1402,NaN,NaN
83,Burma,143629,143751,122,NaN,NaN
85,"Korea, South",140799,141476,677,NaN,NaN
91,Kosovo,107002,107021,19,NaN,NaN


In [22]:
p_merged = pd.merge(pop_latest, country, left_on="country", right_on="Country/Region", how="left")
len(p_merged.loc[p_merged["Country/Region"].isna()])

64

In [23]:
pop_latest.loc[pop_latest["country"].str.contains("Syria")]

,country,population
104,Syrian Arab Republic,17500.657


In [24]:
renames = {
    "US":"United States of America",
    "Russia":"Russian Federation",
    "Iran":"Iran (Islamic Republic of)",
    "Bolivia":"Bolivia (Plurinational State of)",
    "Moldova":"Republic of Moldova",
    "Venezuela":"Venezuela (Bolivarian Republic of)",
    "Korea, South":"Republic of Korea",
    "Congo (Kinshasa)":"Democratic Republic of the Congo",
    "Congo (Brazzaville)":"Congo",
    "Vietnam":"Viet Nam",
    "Laos":"Lao People's Democratic Republic",
    "Taiwan*":"China, Taiwan Province of China",
    "Tanzania":"United Republic of Tanzania",
    "Brunei":"Brunei Darussalam",
    "Micronesia":"Micronesia (Fed. States of)",
    "Syria":"Syrian Arab Republic",
}

In [25]:
country_renamed = country.copy()
country_renamed["Country/Region"].replace(renames,inplace=True)

In [26]:
c_merged = pd.merge(country_renamed, pop_latest, left_on="Country/Region", right_on="country", how="left")
c_merged.loc[c_merged["country"].isna()]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population
59,West Bank and Gaza,308350,308732,382,NaN,NaN
83,Burma,143629,143751,122,NaN,NaN
91,Kosovo,107002,107021,19,NaN,NaN
112,Cote d'Ivoire,47292,47319,27,NaN,NaN
178,Diamond Princess,712,712,0,NaN,NaN
187,MS Zaandam,9,9,0,NaN,NaN


In [27]:
c_merged["positives per 1000 population"] = c_merged["latest_confirmed"] / (c_merged["population"])
c_merged["positives per 100,000 population"] = (c_merged["latest_confirmed"] / (c_merged["population"]/100)).astype(float)
c_merged["new positives per 100,000 population"] = (c_merged["new_confirmed"] / (c_merged["population"]/100)).astype(float)

In [28]:
c_merged = c_merged.sort_values("positives per 1000 population", ascending=False)

In [29]:
c_merged.loc[c_merged["Country/Region"]=="United States of America"]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population,positives per 1000 population,"positives per 100,000 population","new positives per 100,000 population"
0,United States of America,33267507,33290450,22943,United States of America,331002.647,100.574573,10057.457335,6.931365


In [30]:
fig = plt.figure(figsize=(12,8))

plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("positives per pupulation WORST 50")
#plt.xticks(rotation="90",fontsize=9)
plt.yticks(fontsize=9)
plt.xlabel("count per 100,000 population")
plt.grid(axis="x")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
#plt.bar(c_merged.head(50)["Country/Region"],c_merged.head(50)["positives per 100,000 population"],label="positive")
chart_data=c_merged.head(50).sort_values("positives per 100,000 population",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["positives per 100,000 population"],label="positive")
#plt.text(0,chart_data["positives per 100,000 population"],chart_data["population"].astype(str))
#plt.text(0,chart_data["positives per 100,000 population"].all(),chart_data["population"].astype(str).all())
plt.legend()

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("positives per pupulation BEST 50")
#plt.xticks(rotation="90",fontsize=9)
plt.yticks(fontsize=9)
plt.xlabel("count per 100,000 population")
plt.grid(axis="x")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
c_asc = c_merged.sort_values("positives per 1000 population")
chart_data=c_asc.head(50).sort_values("positives per 100,000 population",ascending=False)
plt.barh(chart_data["Country/Region"],chart_data["positives per 100,000 population"],label="positive")
plt.legend()

fig.tight_layout()


<IPython.core.display.Javascript object>

In [31]:
fig.savefig("out/covid-global-tops.png")

In [32]:
plt.figure(figsize=(9.5,5))
plt.title("histgram(positives per 100,000 pop)")
plt.hist(c_merged["positives per 100,000 population"],bins=[0,2000,4000,8000,10000,12000,14000,16000,18000])

<IPython.core.display.Javascript object>

(array([100.,  24.,  37.,  14.,   5.,   2.,   4.,   1.]),
 array([    0,  2000,  4000,  8000, 10000, 12000, 14000, 16000, 18000]),
 <a list of 8 Patch objects>)

In [33]:
c_merged.describe()

,latest_confirmed2,latest_confirmed,new_confirmed,"positives per 100,000 population","new positives per 100,000 population"
count,1.930000e+02,1.930000e+02,193.000000,187.000000,187.000000
mean,8.845814e+05,8.869824e+05,2400.974093,3388.226950,8.211913
std,3.434574e+06,3.443591e+06,11626.114078,3924.246669,18.708277
min,1.000000e+00,1.000000e+00,0.000000,0.852108,0.000000
25%,1.162100e+04,1.162100e+04,5.000000,189.515099,0.064245
50%,9.526300e+04,9.582100e+04,154.000000,1737.345935,1.887138
75%,3.944390e+05,3.950750e+05,1057.000000,6269.733142,8.129629
max,3.326751e+07,3.329045e+07,132788.000000,17768.718048,142.079616


In [34]:
c_merged.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population,positives per 1000 population,"positives per 100,000 population","new positives per 100,000 population"
137,Andorra,13727,13729,2,Andorra,77.265,177.68718,17768.718048,2.588494
95,Montenegro,99623,99652,29,Montenegro,628.062,158.665864,15866.586420,4.617379
20,Czechia,1661272,1661780,508,Czechia,10708.982,155.176281,15517.628099,4.743682
159,San Marino,5090,5090,0,San Marino,33.938,149.979374,14997.937415,0.000000
70,Bahrain,240531,242790,2259,Bahrain,1701.583,142.684782,14268.478235,132.758731


In [35]:
c_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 137 to 187
Data columns (total 9 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Country/Region                        193 non-null    object 
 1   latest_confirmed2                     193 non-null    int64  
 2   latest_confirmed                      193 non-null    int64  
 3   new_confirmed                         193 non-null    int64  
 4   country                               187 non-null    object 
 5   population                            187 non-null    object 
 6   positives per 1000 population         187 non-null    object 
 7   positives per 100,000 population      187 non-null    float64
 8   new positives per 100,000 population  187 non-null    float64
dtypes: float64(2), int64(3), object(4)
memory usage: 15.1+ KB


In [36]:
plt.figure(figsize=(9.5,5))
plt.title("histgram(positives per 100,000 pop)")
plt.hist(c_merged["positives per 100,000 population"],bins=200)

<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/masaru/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([26., 18., 15.,  6.,  2.,  4.,  3.,  1.,  0.,  2.,  0.,  1.,  4.,
         1.,  3.,  2.,  1.,  0.,  4.,  1.,  3.,  3.,  0.,  1.,  2.,  0.,
         3.,  2.,  0.,  1.,  2.,  2.,  0.,  3.,  0.,  0.,  2.,  2.,  1.,
         1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,
         0.,  0.,  1.,  0.,  1.,  0.,  2.,  0.,  0.,  1.,  0.,  1.,  0.,
         1.,  1.,  2.,  0.,  0.,  3.,  2.,  0.,  0.,  1.,  1.,  0.,  1.,
         1.,  1.,  2.,  3.,  0.,  1.,  1.,  2.,  0.,  0.,  3.,  1.,  1.,
         1.,  0.,  2.,  0.,  1.,  1.,  2.,  2.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  2.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.

## 新規感染者数

In [37]:
fig = plt.figure(figsize=(12,8))

plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("新規感染者数 Worst 50({})".format(latest_date))
plt.yticks(fontsize=9)
plt.xlabel("count (people)")
chart_data = country.sort_values("new_confirmed",ascending=False).head(50).sort_values("new_confirmed",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["new_confirmed"],label="positive")
mean = int(country.new_confirmed.mean())
plt.plot([mean,mean],[0,50], 'k--', lw=0.5)
plt.text(mean,50,"平均:{}".format(mean),ha="center",fontsize=9)
plt.legend()

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("人口１０万人当たり新規感染者数 Worst 50({})".format(latest_date))
plt.yticks(fontsize=9)
plt.xlabel("count (people)")
chart_data = c_merged.sort_values("new positives per 100,000 population",ascending=False).head(50).sort_values("new positives per 100,000 population",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["new positives per 100,000 population"],label="positive")
mean = int(c_merged["new positives per 100,000 population"].mean())
plt.plot([mean,mean],[0,50], 'k--', lw=0.5)
plt.text(mean,50,"平均:{}".format(mean),ha="center",fontsize=9)
plt.legend()


fig.tight_layout()

<IPython.core.display.Javascript object>

In [38]:
fig.savefig("out/covid-global-new-positives-latest-top.png")